In [4]:
%pip install -r requirements.txt

  Using cached numpy-2.4.0-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (6.6 kB)
  Using cached tensorflow-2.18.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.1 kB)
  Using cached tflite_runtime-2.14.0-cp311-cp311-manylinux2014_x86_64.whl.metadata (1.4 kB)
  Using cached opencv_python-4.7.0.72-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
  Using cached mediapipe-0.10.31-py3-none-manylinux_2_28_x86_64.whl.metadata (9.7 kB)
  Using cached jupyter-1.1.1-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached absl_py-2.3.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.12.19-py2.py3-none-any.whl.metadata (1.0 kB)
  Using cached gast-0.7.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-manylinux2010_x86_64.whl.metadata (5.2 k

In [ ]:
import cv2

# Open the default camera (0 usuammlly refers to the first camera)
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    if not ret:
        print("Failed to grab frame")
        break
    
    # Display the resulting frame
    cv2.imshow('Camera', frame)
    
    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and close windows
cap.release()
cv2.destroyAllWindows()


In [ ]:
class_names = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'Nothing', 'O', 'P', 'Q', 'R', 'S', 'Space', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']

In [4]:
import cv2
import numpy as np
import mediapipe as mp

import tflite_runtime.interpreter as tflite

class_names = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'Nothing', 'O', 'P', 'Q', 'R', 'S', 'Space', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
TARGET_SIZE = 224
MODEL_PATH = "/home/rafayahmadraza/SignBuddy/Models/asl_model_1.tflite"   # your trained model file


# -------------------------------------------------------
# Resize the cropped hand to 224×224 while keeping aspect
# -------------------------------------------------------
def resize_with_padding(img, target=TARGET_SIZE):
    h, w = img.shape[:2]
    aspect = w / h

    if aspect > 1:  # wide
        new_w = target
        new_h = int(target / aspect)
    else:  # tall
        new_h = target
        new_w = int(target * aspect)

    resized = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_AREA)

    # Add black padding
    top = (target - new_h) // 2
    bottom = target - new_h - top
    left = (target - new_w) // 2
    right = target - new_w - left

    padded = cv2.copyMakeBorder(
        resized, top, bottom, left, right,
        cv2.BORDER_CONSTANT, value=(0, 0, 0)
    )
    return padded


# -------------------------------------------------------
# Load TFLite Model
# -------------------------------------------------------
interpreter = tflite.Interpreter(model_path=MODEL_PATH)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Model input:", input_details[0]["shape"])


# -------------------------------------------------------
# Initialize MediaPipe Hands
# -------------------------------------------------------
mp_draw = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    min_detection_confidence=0.6,
    min_tracking_confidence=0.6
)

cap = cv2.VideoCapture(0)


# -------------------------------------------------------
# Main loop
# -------------------------------------------------------
while True:
    ret, frame = cap.read()
    if not ret:
        break

    h, w, _ = frame.shape

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb)

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:

            # Extract bounding box
            xs = [lm.x for lm in hand_landmarks.landmark]
            ys = [lm.y for lm in hand_landmarks.landmark]

            xmin = int(min(xs) * w)
            xmax = int(max(xs) * w)
            ymin = int(min(ys) * h)
            ymax = int(max(ys) * h)

            pad = 30
            xmin = max(0, xmin - pad)
            ymin = max(0, ymin - pad)
            xmax = min(w, xmax + pad)
            ymax = min(h, ymax + pad)

            # Draw box
            cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)

            # Crop
            crop = frame[ymin:ymax, xmin:xmax]

            if crop.size > 0:
                crop_96 = resize_with_padding(crop)

                # Show resized hand
                cv2.imshow("Hand 96x96", crop_96)

                # Prepare input for model
                inp = cv2.cvtColor(crop_96, cv2.COLOR_BGR2RGB)
                inp = inp.astype(np.float32) / 255.0
                inp = np.expand_dims(inp, axis=0)

                # Run inference
                interpreter.set_tensor(input_details[0]["index"], inp)
                interpreter.invoke()
                prediction = interpreter.get_tensor(output_details[0]["index"])

                # Get predicted class
                cls = np.argmax(prediction)
                conf = np.max(prediction)

                # Display prediction
                text = f"Class: {class_names[np.argmax(prediction)]}  ({conf:.2f})"
                cv2.putText(frame, text, (xmin, ymin - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

                # Draw landmarks
                mp_draw.draw_landmarks(
                    frame, hand_landmarks, mp_hands.HAND_CONNECTIONS
                )

    cv2.imshow("Live", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
I0000 00:00:1765744265.677002  125109 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1765744265.679147  129888 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 25.1.5-1pop0~1753463422~24.04~8af185e), renderer: Mesa Intel(R) UHD Graphics (CML GT2)
W0000 00:00:1765744265.703547  129880 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1765744265.722953  129880 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Model input: [  1 224 224   3]


In [6]:
import cv2
import numpy as np
import mediapipe as mp
from tensorflow.keras.models import load_model

# -------------------------------------------------------
# SETTINGS
# -------------------------------------------------------
class_names = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N',
               'Nothing', 'O', 'P', 'Q', 'R', 'S', 'Space', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z','del']
print(len(class_names))

TARGET_SIZE = 96
MODEL_PATH = "/home/rafayahmadraza/SignBuddy/Models/asl_model_6.h5"   # 👈 now .h5 file


# -------------------------------------------------------
# Resize the cropped hand to 224×224 while keeping aspect
# -------------------------------------------------------
def resize_with_padding(img, target=TARGET_SIZE):
    h, w = img.shape[:2]
    aspect = w / h

    if aspect > 1:  # wide
        new_w = target
        new_h = int(target / aspect)
    else:  # tall
        new_h = target
        new_w = int(target * aspect)

    resized = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_AREA)

    # Add black padding
    top = (target - new_h) // 2
    bottom = target - new_h - top
    left = (target - new_w) // 2
    right = target - new_w - left

    padded = cv2.copyMakeBorder(
        resized, top, bottom, left, right,
        cv2.BORDER_CONSTANT, value=(0, 0, 0)
    )
    return padded


# -------------------------------------------------------
# Load Keras Model
# -------------------------------------------------------
print("Loading Keras model...")
model = load_model(MODEL_PATH)
print("Model loaded!")


# -------------------------------------------------------
# Initialize MediaPipe Hands
# -------------------------------------------------------
mp_draw = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    min_detection_confidence=0.6,
    min_tracking_confidence=0.6
)

cap = cv2.VideoCapture(0)


# -------------------------------------------------------
# Main loop
# -------------------------------------------------------
while True:
    ret, frame = cap.read()
    if not ret:
        break

    h, w, _ = frame.shape

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb)

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:

            # Extract bounding box
            xs = [lm.x for lm in hand_landmarks.landmark]
            ys = [lm.y for lm in hand_landmarks.landmark]

            xmin = int(min(xs) * w)
            xmax = int(max(xs) * w)
            ymin = int(min(ys) * h)
            ymax = int(max(ys) * h)

            pad = 30
            xmin = max(0, xmin - pad)
            ymin = max(0, ymin - pad)
            xmax = min(w, xmax + pad)
            ymax = min(h, ymax + pad)

            # Draw box
            cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)

            # Crop
            crop = frame[ymin:ymax, xmin:xmax]

            if crop.size > 0:
                crop_96 = resize_with_padding(crop)

                # Show resized hand
                cv2.imshow("Hand 224x224", crop_96)

                # Prepare input for model
                inp = cv2.cvtColor(crop_96, cv2.COLOR_BGR2RGB)
                inp = inp.astype(np.float32) / 255.0
                inp = np.expand_dims(inp, axis=0)

                # Predict
                prediction = model.predict(inp)[0]

                # Get predicted class
                cls = np.argmax(prediction)
                conf = prediction[cls]

                # Display prediction
                text = f"Class: {class_names[cls]}  ({conf:.2f})"
                cv2.putText(frame, text, (xmin, ymin - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

                # Draw landmarks
                mp_draw.draw_landmarks(
                    frame, hand_landmarks, mp_hands.HAND_CONNECTIONS
                )
            

    cv2.imshow("Live", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


29
Loading Keras model...


I0000 00:00:1766080902.949868   30210 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1766080902.951658  102747 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 25.1.5-1pop0~1753463422~24.04~8af185e), renderer: Mesa Intel(R) UHD Graphics (CML GT2)
W0000 00:00:1766080902.974823  102742 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1766080902.989356  102738 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Model loaded!
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 989ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/st

In [5]:
import tensorflow as tf

2026-01-03 15:45:58.534113: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-03 15:45:58.537726: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-03 15:45:58.548148: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767437158.562895   48325 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767437158.566824   48325 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2026-01-03 15:45:58.580734: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

AttributeError: _ARRAY_API not found

/home/rafayahmadraza/.pyenv/versions/3.11.8/lib/python3.11/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [8]:
model = tf.keras.models.load_model("Models/asl_model_6.h5")

2026-01-03 15:46:42.034242: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [9]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

In [10]:
tf_lite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpktzmt_8e/assets


INFO:tensorflow:Assets written to: /tmp/tmpktzmt_8e/assets


Saved artifact at '/tmp/tmpktzmt_8e'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 96, 96, 3), dtype=tf.float32, name='input_layer_2')
Output Type:
  TensorSpec(shape=(None, 29), dtype=tf.float32, name=None)
Captures:
  140019586645264: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140019570772688: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140019570773072: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140019586645456: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140019586645840: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140019586645648: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140019570773840: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140019570774032: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140019570772496: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140019570770960: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14001957077364

W0000 00:00:1767437340.246412   48325 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1767437340.246441   48325 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2026-01-03 15:49:00.247034: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpktzmt_8e
2026-01-03 15:49:00.258581: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2026-01-03 15:49:00.258619: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpktzmt_8e
I0000 00:00:1767437340.362482   48325 mlir_graph_optimization_pass.cc:401] MLIR V1 optimization pass is not enabled
2026-01-03 15:49:00.384019: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2026-01-03 15:49:01.108996: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmpktzmt_8e
2026-01-03 15:49:01.298561: I tensorflow/cc/saved_model/loader.cc:466] SavedModel 

In [11]:
with open("Models/ccn_model.tflite", "wb") as f:
    f.write(tf_lite_model)